# Support Vector Machine exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*

In this exercise you will:
    
- implement the fully-vectorized **loss function** for the linear SVM
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**


## SVM Classifier

Your code for this section will be written inside **svm/classifiers/**. 

Please implement the fully-vectorized **loss function** for the linear SVM.

In [1]:
from svm.classifiers.linear_svm import linear_svm_loss_vectorized
import time
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

import matplotlib.pyplot as plt

diabetes = pd.read_csv('svm/datasets/diabetes.csv')
x_train, x_test, y_train, y_test = train_test_split(np.array(diabetes.loc[:, diabetes.columns != 'Outcome']), np.array(diabetes['Outcome']), stratify=np.array(diabetes['Outcome']), random_state=66)
print(x_train.shape, x_test.shape)
# Split the data into train, val, and test sets. In addition we will
# create a small development set as a subset of the training data;
# we can use this for development so our code runs faster.
num_training = 480
num_validation = 96 
num_test = 160
num_dev = 32

# Change label set from {0, 1} to {-1, 1}
y_train = 2 * y_train - 1
y_test = 2 * y_test - 1

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = x_train[mask]
y_val = y_train[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = x_train[mask]
y_train = y_train[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = x_train[mask]
y_dev = y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = x_test[mask]
y_test = y_test[mask]


 
# generate a random SVM weight matrix of small numbers
W = np.random.randn(9, ) * 0.0001 
#### Evaluate the implementation of the linear SVM loss we provided for you:
loss, grad = linear_svm_loss_vectorized(W, X_dev, y_dev, 0.00001)

#print scores.shape
print('loss: %f' % (loss, ))


(576, 8) (192, 8)
loss: 1.000522
y_train(part): 
[ 1 -1  1  1 -1  1 -1 -1  1 -1]


The `grad` returned from the function above is right now all zero. Derive and implement the gradient for the SVM cost function and implement it inline inside the function `linear_svm_loss_vectorized`. You will find it helpful to interleave your new code inside the existing function.


### Stochastic Gradient Descent

We now have vectorized and efficient expressions for the loss, the gradient. We are therefore ready to do SGD to minimize the loss of the SVM.

In [2]:
# In the file svm_classifier.py, implement SGD in the function
# SVMClassifier.train() and then run it with the code below.
from svm.classifiers import LinearSVM
svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=5e4,
                      num_iters=1500, verbose=True)

toc = time.time()
print('That took %fs' % (toc - tic))
# print('loss history: ')
# print(loss_hist)

91129, 0.9626569159384498, 0.9595011670982201, 0.9617013725720588, 0.9645827490030213, 0.9612234257809873, 0.9566012940540647, 0.9517684353325011, 0.9689549498966423, 0.9558177738853074, 0.9428387400525803, 0.9585924867337439, 0.9520296473746283, 0.9562440719111345, 0.960264717440944, 0.9566045733702256, 0.95988305399426, 0.9573996991641363, 0.9600814029558059, 0.960917548323429, 0.9582183102267743, 0.9555976262953007, 0.9577468308009852, 0.9549303440000587, 0.9556533142670804, 0.9532393287305239, 0.9621931359649126, 0.9539677159020549, 0.959058417772372, 0.9578315432272281, 0.9524682084021658, 0.9488897501795657, 0.9660176646916203, 0.9550122358949293, 0.961404250988569, 0.9567785830375332, 0.9523450397078922, 0.9515328005457223, 0.9632576631296199, 0.9492007090391479, 0.9555502417071854, 0.9500180990328131, 0.964563634764923, 0.9600646028034506, 0.950310322127368, 0.9577893771787988, 0.9556394937429248, 0.9613673468168834, 0.962706233053054, 0.9519145387399586, 0.9560792048103637, 0.

In [7]:
# Write the SVMClassifier.predict function and evaluate the performance on both the training and validation set
y_train_pred = svm.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = svm.predict(X_val)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

training accuracy: 0.000000
validation accuracy: 0.000000


/Users/yaozy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yaozy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """


In [8]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths.

best_val = -1   # The highest validation accuracy that we have seen so far.
best_svm = None # The SVM object that achieved the highest validation rate.

################################################################################
# TODO:                                                                        #
# Write code that chooses the best hyperparameters by tuning on the validation #
# set. For each combination of hyperparameters, train an SVM on the            #
# training set, compute its accuracy on the training and validation sets.      #
# In addition, store the best                                                  #
# validation accuracy in best_val and the SVM object that achieves this        #
# accuracy in best_svm.                                                        #
#                                                                              #
# Hint: You should use a small value for num_iters as you develop your         #
# validation code so that the SVMs don't take much time to train; once you are #
# confident that your validation code works, you should rerun the validation   #
# code with a larger value for num_iters.                                      #
################################################################################






################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
# Your code
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

best validation accuracy achieved during cross-validation: -1.000000


In [9]:
# Evaluate the best linear SVM on test set
# Your code